# Model 1: Fully Connected Neural Network

In [ ]:
from pathlib import Path
import numpy as np
import torch 

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

In [ ]:
# Import local modules from 'src/utils' as package 'utils'
import sys; sys.path.insert(0, '../')

## Read Datasets from .csv

In [ ]:
from utils.file_io import read_angle_datasets

In [ ]:
data_folder = Path("../../data/")
train_data, test_data = read_angle_datasets(data_folder, 0.9)

In [ ]:
input_shape, output_shape = train_data[0][0].shape[0], train_data[0][1].shape[0]
print(f"Data shape {input_shape} / {output_shape} of total {len(train_data)} data rows!")

## Model definition

In [ ]:
from torch import nn, Tensor
from typing import Tuple, List

In [ ]:
class FullyConnected(nn.Module):
    def __init__(self, flattened_input_dim: int, intermediate_dims: List[int], output_dim: int, dropout: float = 0.25, hidden_activation = nn.ReLU) -> None:
        super().__init__()
        self.total_epochs = 0
        self.flatten = nn.Flatten()
        self.hidden = nn.Sequential()
        for i, dim in enumerate(intermediate_dims):
            if i == 0:
                self.hidden.add_module(f"linear_{i+1}", nn.Linear(flattened_input_dim, dim))
            else:
                self.hidden.add_module(f"linear_{i+1}", nn.Linear(intermediate_dims[i-1], dim))

            self.hidden.add_module(f"hidden_activation_{i+1}", hidden_activation())
            self.hidden.add_module(f"dropout_{i+2}", nn.Dropout(dropout))

        self.last = nn.Linear(intermediate_dims[-1], output_dim)

    def forward(self, x: Tensor) -> Tensor:
        x = self.flatten(x)
        x = self.hidden(x)
        return self.last(x)

In [ ]:
model = FullyConnected(input_shape, [32, 16], output_shape)

## Load parameter, functions and dataloader

In [ ]:
import os
from dotenv import load_dotenv
from torch.utils.data import DataLoader

from utils.file_io import save_model
from utils.evaluation import compute_loss_on
from utils.file_io import define_dataloader_from_angle_dataset

In [ ]:
model_path = Path("../../models/fully_connected/")

In [ ]:
dotenv_path = model_path / ".env"
load_dotenv(dotenv_path=dotenv_path)

learning_rate = float(os.getenv("LEARNING_RATE"))
batch_size = int(os.getenv("BATCH_SIZE"))
num_epochs = int(os.getenv("NUM_EPOCHS"))

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_function = torch.nn.MSELoss()

In [ ]:
train_dataloader, validation_dataloader, test_dataloader = define_dataloader_from_angle_dataset(train_data, test_data, batch_size, split_size=0.95)

## Define train methods

In [ ]:
def train_epoch(train_dataloader: DataLoader, model: nn.Module, loss_function, optimizer, 
                device: torch.device, report_interval: int = 1000) -> float:

    running_loss = 0
    last_loss = 0
    
    for i, (inputs, true_values) in enumerate(train_dataloader):

        inputs = inputs.to(device)
        true_values = true_values.to(device)
                
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, true_values)
        running_loss += loss
        loss.backward()
        optimizer.step() 
    
    if i % report_interval == report_interval - 1:
        last_loss = running_loss / report_interval
        print(f"batch {i + 1}, Mean Squared Error: {last_loss}")
        running_loss = 0
    
    return last_loss 

In [ ]:
def train(epochs: int, train_dataloader: DataLoader, validation_dataloader: DataLoader, model: nn.Module, loss_function, optimizer, 
          checkpoint_path: Path, device: torch.device = 'cpu', report_interval: int = 1000) -> nn.Module:
    
    best_val_loss = float("inf")

    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    model.to(device)

    for epoch in range(model.total_epochs, epochs):
        print(f"Epoch: {epoch + 1}")

        model.train(True)
        avg_loss = train_epoch(train_dataloader, model, loss_function, optimizer, device, report_interval)
        model.eval()

        with torch.no_grad():
            avg_val_loss = compute_loss_on(validation_dataloader, model, loss_function, device=device)

        print(f"Loss on train: {avg_loss}, loss on validation: {avg_val_loss}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            model_path = checkpoint_path / f"{checkpoint_path.name}_{epoch}.pt"
            save_model(model, model_path)

        model.total_epochs += 1
    
    return model   

## Train the model

In [ ]:
last_model = train(num_epochs, train_dataloader, validation_dataloader, model, loss_function, optimizer, model_path, device=device)

## Evaluation

In [ ]:
from utils.evaluation import compute_predictions, compute_losses_from
from utils.visualization import create_trace_animation
from matplotlib import pyplot as plt

from IPython.display import HTML

In [ ]:
# Compute evaluation on the cpu
device = 'cpu'

### Compute mean squared error


In [ ]:
y, y_true = compute_predictions(test_dataloader, last_model, device=device)
test_losses = compute_losses_from(y, y_true, loss_function)
print(f"The mean squared error on test is: {test_losses.mean()}")

### Draw prediction/truth traces 

In [ ]:
%matplotlib notebook
 
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  

In [ ]:
animation = create_trace_animation(y.numpy(), y_true.numpy())
HTML(animation.to_jshtml())

## Loading the best model

In [ ]:
from utils.file_io import load_model

In [ ]:
loaded_model = FullyConnected(input_shape, [32, 16], output_shape)
model_state_dict = load_model(model_path)
loaded_model.load_state_dict(model_state_dict)
loaded_model.eval()
y, y_true = compute_predictions(test_dataloader, loaded_model, device)
test_losses = compute_losses_from(y, y_true, loss_function)
print(f"The mean squared error of the loaded model on test is: {test_losses.mean()}")
animation = create_trace_animation(y.numpy(), y_true.numpy())
HTML(animation.to_jshtml())